In [4]:
import llm_class
import output_parser
import phrase_extraction_evaluation
from model_eval import Model_eval
import keyphrase_dataset
import model_inference

model = llm_class.LanguageModel()
keyphrase_set = keyphrase_dataset.KeyphraseDataset()
inference = model_inference.ModelInferencing(model)


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import gradio as gr
import prompts

def gradio_func(text_prompt, samples,experiment_name):
    prompts.multiple_keyphrases_prompt=text_prompt
    tests = keyphrase_set.get_samples(samples)
    results = Model_eval.multiple_samples(inference, tests, save_file="default", print_counts=True)
    
    return f"Avg cosine: {results['cosine']} | Avg labels matched: {results['matchings']} | Avg redundancy: {results['redundancy']} | Avg groundness: {results['groundness']}"

# Create the Gradio interface
interface = gr.Interface(
    fn=gradio_func,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your text prompt here...", label="Text Prompt"),
        gr.Slider(minimum=1, maximum=500, step=1, label="Number of tests"),
        gr.Textbox(lines=2, placeholder="Enter your text prompt here...", label="Experiment name"),
    ],
    outputs="text",
    title="Prompt testing interface",
)

# Launch the interface
interface.launch(share=True)

In [11]:
# Debug

model.set_prompt("""You are provided with the text extracted from a webpage, delimited by < for start and > for end. Your task is to extract the key phrases from the text that best characterize the webpage. You should extract at most 10 such phrases, but may extract less. Ensure the key phrases are relevant and provide a good summary of the content. Present the key phrases in JSON format, with each key phrase being an item in a list. Do not output anything but json of extracted keyphrases.

Example webpage to extract from: <&>

Your response should look like this:&

The text from the webpage: <&>""")

In [6]:
# Evaluating prompts with gradio

import gradio as gr
from datetime import datetime


def visualize(text):
    formated_phrases=inference.get_multiple_phrases(text)
    output_text="Key phrases in the text are:\n- " + "\n- ".join(formated_phrases)
    
    return output_text

# Create the Gradio interface
interface = gr.Interface(
    fn=visualize,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your text prompt here...", label="Text to extract the keywords from"),
    ],
    outputs="text",
    title="Key phrase extractor",
)

# Launch the interface
interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://4980806db5e7064156.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Debug

sample=keyphrase_set.train_set[1]

#print(model.prompt_text)

print(sample['text'])
print("\n")
print(sample['label'])
print("\n")
print(get_metrics(sample['text'],sample['label']))